You have to work on the University dataset. It contains three different university rankings:

The Times Higher Education World University Ranking, shortly Times,
the Academic Ranking of World Universities, shortly Shanghai,
the Center for World University Rankings, shortly cwur.
Notes
It is mandatory to use GitHub for developing the project.
The project must be a jupyter notebook.
There is no restriction on the libraries that can be used, nor on the Python version.
All questions on the project must be asked in a public channel on Zulip.
1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes
2. For each university, compute the improvement in income between the least recent and the most recent data points
3. Find the university with the largest increase computed in the previous point
4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range
5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.
6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)
7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).
8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name
9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value
10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).
11. In the times ranking, compute the number of times each university appears
12. Find the universities that appear at most twice in the times ranking.
13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [1]:
import pandas as pd

### 1) For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [2]:
times = pd.read_csv('timesData.csv')
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


**Check se la coppia [university_name,year] è chiave**

In [3]:
times.groupby([times.university_name, times.year]).year.count().value_counts()

1    2603
Name: year, dtype: int64

**Aggregazione per università, estraggo min a max da anno**

In [4]:
yearMinMax = times.groupby(times.university_name).agg({'year' :['min', 'max']})
yearMinMax

year      
                                           min   max
university_name                                     
AGH University of Science and Technology  2016  2016
Aalborg University                        2012  2016
Aalto University                          2012  2016
Aarhus University                         2011  2016
Aberystwyth University                    2012  2016
...                                        ...   ...
École Normale Supérieure                  2011  2016
École Normale Supérieure de Lyon          2011  2016
École Polytechnique                       2011  2016
École Polytechnique Fédérale de Lausanne  2011  2016
Örebro University                         2016  2016

[818 rows x 2 columns]

In [5]:
yearMinMax[yearMinMax.year['min'] == yearMinMax.year['max']]

year      
                                           min   max
university_name                                     
AGH University of Science and Technology  2016  2016
Adam Mickiewicz University                2016  2016
Aix-Marseille University                  2016  2016
Ajou University                           2016  2016
Alexandru Ioan Cuza University            2016  2016
...                                        ...   ...
Yeungnam University                       2016  2016
Yokohama City University                  2016  2016
Yokohama National University              2016  2016
Yıldız Technical University               2016  2016
Örebro University                         2016  2016

[344 rows x 2 columns]

**Abbiamo 818 Università, di cui 344 con un solo anno, quindi se prendiamo solo le righe con anno min e max dal DataFrame 'times' ci aspettiamo <br> 818x2=1636 record con 344 duplicati, dopo averli rimossi dovrebbe rimanere un DataFrame con 1636-344=1292 righe di valori unici**

In [6]:
yearMinMax.columns = yearMinMax.columns.droplevel(0)
yearMinMax = yearMinMax.reset_index()
yearMinMax

,university_name,min,max
0,AGH University of Science and Technology,2016,2016
1,Aalborg University,2012,2016
2,Aalto University,2012,2016
3,Aarhus University,2011,2016
4,Aberystwyth University,2012,2016
...,...,...,...
813,École Normale Supérieure,2011,2016
814,École Normale Supérieure de Lyon,2011,2016
815,École Polytechnique,2011,2016
816,École Polytechnique Fédérale de Lausanne,2011,2016


In [7]:
minYearInfo = pd.merge(times, yearMinMax,  how='inner', left_on=['university_name','year'], right_on = ['university_name','min'])
maxYearInfo = pd.merge(times, yearMinMax,  how='inner', left_on=['university_name','year'], right_on = ['university_name','max'])

In [8]:
print(len(minYearInfo))
print(len(maxYearInfo))
timesMinMaxYear = pd.concat([minYearInfo,maxYearInfo]).iloc[:, :-2]
print(len(timesMinMaxYear))
timesMinMaxYear = timesMinMaxYear.drop_duplicates()
print(len(timesMinMaxYear))

818
818
1636
1292


##### Come ci aspettavamo

In [9]:
timesMinMaxYear

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,601-800,Polytechnic University of Valencia,Spain,20.3,32.9,12.7,34.3,43.8,-,"33,172",13.6,7%,36 : 64,2016
765,601-800,University of Seoul,South Korea,22.8,18.7,17.5,21.4,28.9,-,"10,243",19.0,3%,36 : 64,2016
804,601-800,University of Vigo,Spain,18.4,30.7,10.5,31.8,38.1,-,"22,793",19.0,3%,51 : 49,2016
807,601-800,Waseda University,Japan,23.6,29.7,14.6,29.4,32.4,-,"52,316",16.9,8%,35 : 65,2016


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

**Setto i valori stringa '-' a Null nel campo 'income'**

In [10]:
timesMinMaxYear.income = pd.to_numeric(timesMinMaxYear.income, errors='coerce')
query1 = timesMinMaxYear[timesMinMaxYear.income.notna()]

**Estraggo Università con un solo anno**

In [11]:
newEntries = pd.Series(timesMinMaxYear.groupby(timesMinMaxYear.university_name).year.count().where(lambda x : x==1).dropna().index)

In [12]:
newEntries

0      AGH University of Science and Technology
1                    Adam Mickiewicz University
2                      Aix-Marseille University
3                               Ajou University
4                Alexandru Ioan Cuza University
                         ...                   
339                         Yeungnam University
340                    Yokohama City University
341                Yokohama National University
342                 Yıldız Technical University
343                           Örebro University
Name: university_name, Length: 344, dtype: object

**Escludiamo le Università con una sola entries, abbiamo bisogno di almeno 2 anni**

In [13]:
query2 = timesMinMaxYear[~timesMinMaxYear.university_name.isin(newEntries)]
query2

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,"7,929",8.4,27%,45 : 55,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,601-800,Polytechnic University of Valencia,Spain,20.3,32.9,12.7,34.3,43.8,-,"33,172",13.6,7%,36 : 64,2016
765,601-800,University of Seoul,South Korea,22.8,18.7,17.5,21.4,28.9,-,"10,243",19.0,3%,36 : 64,2016
804,601-800,University of Vigo,Spain,18.4,30.7,10.5,31.8,38.1,-,"22,793",19.0,3%,51 : 49,2016
807,601-800,Waseda University,Japan,23.6,29.7,14.6,29.4,32.4,-,"52,316",16.9,8%,35 : 65,2016


In [14]:
query2[query2.university_name == 'École Polytechnique']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,"2,429",4.8,30%,18 : 82,2011
119,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016


In [15]:
query2 = query2.groupby([query2.university_name, query2.year]).income.first()  
query2

university_name                           year
Aalborg University                        2012    36.4
                                          2016    43.7
Aalto University                          2012    61.9
                                          2016    61.6
Aarhus University                         2011    61.5
                                                  ... 
École Normale Supérieure de Lyon          2016    31.7
École Polytechnique                       2011     NaN
                                          2016    82.3
École Polytechnique Fédérale de Lausanne  2011    38.0
                                          2016    65.4
Name: income, Length: 948, dtype: float64

In [16]:
mostRecentIncome = query2.groupby(level=0).apply(max)
leastRecentIncome = query2.groupby(level=0).apply(min)

In [17]:
incomeTable = pd.concat([leastRecentIncome, mostRecentIncome], axis=1).reset_index()
incomeTable.columns = ['university_name','leastRecentIncome', 'mostRecentIncome']
incomeTable['improvement'] = incomeTable['mostRecentIncome'] - incomeTable['leastRecentIncome']
incomeTable

,university_name,leastRecentIncome,mostRecentIncome,improvement
0,Aalborg University,36.4,43.7,7.3
1,Aalto University,61.6,61.9,0.3
2,Aarhus University,61.5,68.3,6.8
3,Aberystwyth University,31.3,35.5,4.2
4,Alexandria University,29.7,36.0,6.3
...,...,...,...,...
469,Zhejiang University,70.3,96.2,25.9
470,École Normale Supérieure,30.7,37.1,6.4
471,École Normale Supérieure de Lyon,26.1,31.7,5.6
472,École Polytechnique,82.3,82.3,0.0


## 3) Find the university with the largest increase computed in the previous point

In [18]:
incomeTable.sort_values(by='improvement', ascending=False)[['university_name','improvement']].head(1)

,university_name,improvement
226,TU Dresden,67.8


## 4) For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

**Carichiamo i dataset relativi alle altre due università**

In [19]:
cwur = pd.read_csv('cwurData.csv')
shanghai = pd.read_csv('shanghaiData.csv')

**Estraiamo dai tre dataset solo le informazioni che ci interessano, avendo cura di uniformare i nomi delle colonne**

In [20]:
timesWR = times[['university_name','year','world_rank']]
cwurWR = cwur[['institution','year','world_rank']].rename(columns={"institution": "university_name"})
shanWR = shanghai[['university_name','year','world_rank']]

**Raggruppiamo per world_rank e per ogni gruppo estraiamo l'anno più recente, <br>
  fatto ciò utilizziamo quanto ottenuto per filtrare i corrispettivi dataset eliminando così i dati meno recenti**

In [21]:
timesWR = timesWR.groupby(timesWR.world_rank).agg({'year' : 'max'}).reset_index().merge(timesWR, how='left', on=['world_rank','year'])
cwurWR = cwurWR.groupby(cwurWR.world_rank).agg({'year' : 'max'}).reset_index().merge(cwurWR, how='left', on=['world_rank','year'])
shanWR = shanWR.groupby(shanWR.world_rank).agg({'year' : 'max'}).reset_index().merge(shanWR, how='left', on=['world_rank','year'])

**Prima di fare la merge bisogna fare pre-processing dei world-rank e riportarli a interi, <br> visualizziamo tutti i possibili rank per vedere in quanti modi diversi si presentano**

In [22]:
print('timesWR: \t',timesWR.world_rank.dtypes,
      '\ncwurWR: \t',cwurWR.world_rank.dtypes,
      '\nshanWR: \t',shanWR.world_rank.dtypes)

timesWR: 	 object 
cwurWR: 	 int64 
shanWR: 	 object


**cwurWR:** Contiene già solo dei numeri interi, non necessita di ulteriori manipolazioni. <br><br>

In [23]:
print(shanWR.world_rank.unique())
timesWR.world_rank.unique()

['1' '10' '100' '101-150' '101-151' '101-152' '102-150' '11' '12' '13'
 '14' '15' '151-200' '151-202' '152-200' '153-202' '16' '17' '18' '19' '2'
 '20' '201-300' '201-302' '203-300' '203-304' '21' '22' '23' '24' '25'
 '26' '27' '28' '29' '3' '30' '301-400' '303-401' '305-402' '31' '32' '33'
 '34' '35' '36' '37' '38' '39' '4' '40' '401-500' '402-501' '402-503'
 '403-510' '41' '42' '43' '44' '45' '46' '47' '48' '49' '5' '50' '51' '52'
 '53' '54' '55' '56' '57' '58' '59' '6' '60' '61' '62' '63' '64' '65' '66'
 '67' '68' '69' '7' '70' '71' '72' '73' '74' '75' '76' '77' '78' '79' '8'
 '80' '81' '82' '83' '84' '85' '86' '87' '88' '89' '9' '90' '91' '92' '93'
 '94' '95' '96' '97' '98' '99']


array(['1', '10', '100', '101', '102', '103', '104', '105', '106', '107',
       '108', '109', '11', '110', '111', '112', '113', '114', '115',
       '116', '117', '118', '119', '12', '120', '121', '122', '123',
       '124', '125', '126', '127', '128', '129', '13', '130', '131',
       '132', '133', '134', '135', '136', '137', '138', '139', '14',
       '140', '141', '142', '143', '144', '145', '146', '147', '148',
       '149', '15', '150', '151', '152', '153', '154', '155', '156',
       '157', '158', '159', '16', '160', '161', '162', '163', '164',
       '165', '166', '167', '168', '169', '17', '170', '171', '172',
       '173', '174', '175', '176', '177', '178', '179', '18', '180',
       '181', '182', '183', '184', '185', '186', '187', '188', '189',
       '19', '190', '191', '192', '193', '194', '195', '196', '197',
       '198', '199', '2', '20', '200', '201-225', '201-250', '21', '22',
       '226-250', '23', '24', '25', '251-275', '251-300', '26', '27',
       '276-300', '28'

**shanWR:** Contiene stringhe direttamente riconducibili a int oppure valori su range <br>
  del formato 'xxx-yyy', questi range verranno riportati all'intero più vicino alla media dei due valori. <br> <br>
**timesWR:** Presenta gli stessi casi di shanWR, ed inoltre una forma '=xxx' nel caso due o più università <br>
  abbiamo lo stesso rank, in tal caso rimuoviamo semplicemente l'uguale e manteniamo il valore del rank intatto<br> <br>
  Definiamo una semplice funzione da applicare alle righe dei dataset per trattarli come descritto sopra:

In [24]:
def RankToInt(mystr):
    if mystr[0] == '=':
        return(int(mystr[1:]))
    elif len(mystr) == 7:
        return(int((int(mystr[0:3]) + int(mystr[4:7]))/2))
    else:
        return(int(mystr))

In [25]:
timesWR.world_rank = timesWR.world_rank.apply(lambda x: RankToInt(x))
shanWR.world_rank = shanWR.world_rank.apply(lambda x: RankToInt(x))

**Dopo aver corretto i world_rank mergiamo i risultati delle tre università utilizzando delle outer join, così da avere tutti i dati a nostra disposizione**

In [26]:
allWR = timesWR.merge(cwurWR, on=['university_name','year'], how='outer').merge(shanWR, on=['university_name','year'], how='outer')
allWR[allWR.university_name == 'Aarhus University']

,world_rank_x,year,university_name,world_rank_y,world_rank
1013,106.0,2016,Aarhus University,NaN,NaN
1187,NaN,2015,Aarhus University,122.0,73.0
2023,NaN,2005,Aarhus University,NaN,126.0
3206,NaN,2014,Aarhus University,NaN,74.0
3218,NaN,2013,Aarhus University,NaN,81.0


**fatto ciò ci basta calcolare la massima differenza per ogni riga del dataframe finora ottenuto come max-min, <br> le righe contenenti un solo record otterranno differenza 0, verranno quindi scartate**

In [27]:
allWR['max_diff'] = allWR.iloc[:, [0,3,4]].max(axis = 1) - allWR.iloc[:, [0,3,4]].min(axis = 1)
allWR = allWR[allWR.max_diff > 0].iloc[:,[2,-1]]
allWR.head()

,university_name,max_diff
4,Maastricht University,149.0
7,University of Helsinki,44.0
8,University of Warwick,201.0
9,University of Zurich,49.0
14,Queen Mary University of London,102.0


**Controlliamo se il risultato è corretto:**

In [28]:
allWR[allWR.university_name == 'Aarhus University']

,university_name,max_diff
1187,Aarhus University,49.0


## 5) Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [29]:
times.head(3)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011


**Prendo il data point più recente**

In [30]:
query5 = times.groupby(['country','university_name']).agg({'year' : 'max'}).reset_index()
query5 = query5.merge(times, how='left', on=['country','university_name','year'])
query5.head()

,country,university_name,year,world_rank,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio
0,Argentina,National University of Córdoba,2016,601-800,16.0,28.1,9.0,12.5,28.6,-,"108,373",38.1,2%,62 : 38
1,Australia,Australian National University,2016,52,54.7,93.3,77.3,72.3,48.0,69.5,"14,604",19.2,35%,52 : 48
2,Australia,Central Queensland University,2016,501-600,20.9,50.8,11.4,43.7,32.5,-,"11,452",29.6,27%,59 : 41
3,Australia,Charles Darwin University,2016,251-300,17.8,63.7,22.6,85.9,43.2,-,"5,570",25.4,15%,67 : 33
4,Australia,Curtin University,2016,401-500,20.1,93.1,20.2,44.6,37.8,-,"30,333",17.1,38%,53 : 47


**Drop dei valori null e SELECT dei valori di interesse**

In [31]:
query5 = query5.dropna(axis=0, subset=['female_male_ratio'])[['country','university_name','num_students','female_male_ratio']]
query5.head()

,country,university_name,num_students,female_male_ratio
0,Argentina,National University of Córdoba,"108,373",62 : 38
1,Australia,Australian National University,"14,604",52 : 48
2,Australia,Central Queensland University,"11,452",59 : 41
3,Australia,Charles Darwin University,"5,570",67 : 33
4,Australia,Curtin University,"30,333",53 : 47


**Check dtypes**

In [32]:
query5.dtypes

country              object
university_name      object
num_students         object
female_male_ratio    object
dtype: object

**Dobbiamo fare un cast di num_student a int e definire una funzione per estrarre <br>
  il numero di studenti maschi o femmine**

In [33]:
query5['num_students'] = query5['num_students'].str.replace(',', '').astype(int)

In [34]:
query5.female_male_ratio.unique()

array(['62 : 38', '52 : 48', '59 : 41', '67 : 33', '53 : 47', '60 : 40',
       '61 : 39', '63 : 37', '55 : 45', '56 : 44', '54 : 46', '47 : 53',
       '66 : 34', '44 : 56', '48 : 52', '57 : 43', '50 : 50', '22 : 78',
       '65 : 35', '27 : 73', '34 : 66', '69 : 31', '70 : 30', '1 : 99',
       '49 : 51', '46 : 54', '58 : 42', '45 : 55', '24 : 76', '39 : 61',
       '33 : 67', '29 : 71', '43 : 57', '25 : 75', '40 : 60', '37 : 63',
       '78 : 22', '51 : 49', '38 : 62', '32 : 68', '21 : 79', '36 : 64',
       '31 : 69', '41 : 59', '-', '23 : 77', '64 : 36', '28 : 72',
       '18 : 82', '26 : 74', '42 : 58', '17 : 83', '19 : 81', '16 : 84',
       '14 : 86', '13 : 87', '15 : 85', '30 : 70', '35 : 65', '11 : 89',
       '20 : 80', '9 : 91', '68 : 32', '71 : 29', '74 : 26', '100 : 0',
       '12 : 88', '76 : 24', '77 : 23'], dtype=object)

In [35]:
query5[query5.female_male_ratio == '-']

,country,university_name,num_students,female_male_ratio
139,Czech Republic,Czech Technical University in Prague,22751,-
405,Poland,AGH University of Science and Technology,35569,-
447,Saudi Arabia,King Fahd University of Petroleum and Minerals,6898,-


**Tre Università presentano un valore nullo per il ratio, eliminiamo queste righe**

In [36]:
query5 = query5.drop([139, 405, 447])

**Definiamo la funzione per estrarre il numero di studenti del genere che ci interessa**

**We will use this function, we have to pass the total number of students, the ratio and the gender we are interested in**

In [37]:
def femMaleCalc(num_stud, ratio, gender):
    if gender == 'male':
        return int(num_stud*(int(ratio.split(':')[1]))/100)
    else:
        return int(num_stud*(int(ratio.split(':')[0]))/100)

In [38]:
query5['male_students'] = query5.apply(lambda x: femMaleCalc(x.num_students, x.female_male_ratio, 'male'), axis=1)
query5['female_students'] = query5.apply(lambda x: femMaleCalc(x.num_students, x.female_male_ratio, 'female'), axis=1)

In [39]:
query5.groupby(query5.country).agg({'num_students'    : 'sum',
                                    'male_students'   : 'sum',
                                    'female_students' : 'sum'}).reset_index().sort_values(by = 'num_students', ascending = False)

,country,num_students,male_students,female_students
69,United States of America,3157775,1556253,1601393
68,United Kingdom,1324842,612993,711775
28,Italy,1129167,486472,642666
9,China,1015150,587297,427827
18,Germany,918502,463855,454615
...,...,...,...,...
36,Macau,8521,3578,4942
33,Lebanon,7695,3770,3924
44,Oman,7089,3544,3544
11,Cyprus,6880,2132,4747


## 6) Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [40]:
avgRatio = round(query5.female_students.sum() / query5.male_students.sum(), 2)
print('Average Female to Male Ratio: ', avgRatio)

Average Female to Male Ratio:  1.07


In [41]:
query6 = query5.copy()

query6['female_to_male'] = query6.apply(lambda x: x.female_students / x.male_students if x.male_students > 0 else None, axis=1)
query6

,country,university_name,num_students,female_male_ratio,male_students,female_students,female_to_male
0,Argentina,National University of Córdoba,108373,62 : 38,41181,67191,1.631602
1,Australia,Australian National University,14604,52 : 48,7009,7594,1.083464
2,Australia,Central Queensland University,11452,59 : 41,4695,6756,1.438978
3,Australia,Charles Darwin University,5570,67 : 33,1838,3731,2.029924
4,Australia,Curtin University,30333,53 : 47,14256,16076,1.127666
...,...,...,...,...,...,...,...
813,United States of America,Washington State University,24550,50 : 50,12275,12275,1.000000
815,United States of America,Wayne State University,23065,57 : 43,9917,13147,1.325703
816,United States of America,William & Mary,7867,54 : 46,3618,4248,1.174129
817,United States of America,Yale University,11751,50 : 50,5875,5875,1.000000


In [42]:
query6[query6.female_to_male < avgRatio].sort_values(by = 'female_to_male', ascending = False)

,country,university_name,num_students,female_male_ratio,male_students,female_students,female_to_male
812,United States of America,Wake Forest University,7326,51 : 49,3589,3736,1.040958
749,United States of America,"University of California, Riverside",20626,51 : 49,10106,10519,1.040867
807,United States of America,University of Wisconsin-Madison,39655,51 : 49,19430,20224,1.040865
393,New Zealand,University of Canterbury,12187,51 : 49,5971,6215,1.040864
121,China,Sun Yat-sen University,51351,51 : 49,25161,26189,1.040857
...,...,...,...,...,...,...,...
262,India,Indian Institute of Technology Kanpur,6167,13 : 87,5365,801,0.149301
490,Spain,Polytechnic University of Catalonia,22893,12 : 88,20145,2747,0.136361
332,Japan,Kyushu Institute of Technology,5890,11 : 89,5242,647,0.123426
355,Japan,Toyohashi University of Technology,2153,9 : 91,1959,193,0.098520


## 7) For each country, compute the fraction of the students in the country that are in one of the universities <br> computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

**Copiamo il dataset precedente e creiamo una nuova colonna che ci indica <br>
  se una data università si trova sotto o sopra la media**

In [43]:
query7 = query6.copy().reset_index()
query7['ratioBelowMean'] = False
query7.at[query7[query7.female_to_male < avgRatio].index,'ratioBelowMean'] = True
query7

,index,country,university_name,num_students,female_male_ratio,male_students,female_students,female_to_male,ratioBelowMean
0,0,Argentina,National University of Córdoba,108373,62 : 38,41181,67191,1.631602,False
1,1,Australia,Australian National University,14604,52 : 48,7009,7594,1.083464,False
2,2,Australia,Central Queensland University,11452,59 : 41,4695,6756,1.438978,False
3,3,Australia,Charles Darwin University,5570,67 : 33,1838,3731,2.029924,False
4,4,Australia,Curtin University,30333,53 : 47,14256,16076,1.127666,False
...,...,...,...,...,...,...,...,...,...
734,813,United States of America,Washington State University,24550,50 : 50,12275,12275,1.000000,True
735,815,United States of America,Wayne State University,23065,57 : 43,9917,13147,1.325703,False
736,816,United States of America,William & Mary,7867,54 : 46,3618,4248,1.174129,False
737,817,United States of America,Yale University,11751,50 : 50,5875,5875,1.000000,True


**Adesso creiamo 2 dataset temporanei, nel primo raggruppiamo le Università per nazione calcolando il numero totale di studenti**

In [44]:
df1 = pd.DataFrame(query7.groupby(query7.country).num_students.sum())
df1.head()

,num_students
country,
Argentina,108373
Australia,713376
Austria,134477
Bangladesh,62716
Belarus,29303


**Il secondo dataframe farà lo stesso calcolo ma solo sul subset del dataset che presentano un ratio female_to_male sotto la media**

In [45]:
df2 = pd.DataFrame(query7[query7.ratioBelowMean == True].groupby(query7.country).num_students.sum())
df2.head()

,num_students
country,
Australia,146235
Austria,61033
Bangladesh,62716
Brazil,278774
Canada,86779


**Uniamo i dataset per ottenere tutte le informazioni di cui necessitiamo. I valori NAN all'interno del df2 sono riempiti con 0, calcoliamo così la frazione dividendo le due colonne ottenute, infine mostriamo il risultato mantenendo solo le informazioni essenziali riguardo la nazione, il numero totale di studenti e la frazione di studenti in Università con ratio fem_to_mal sotto la media**

In [46]:
df1['belowRatio'] = df2['num_students']
df1 = df1.fillna(0)
df1['fractionBelowRatio'] = round(df1['belowRatio'] / df1['num_students'] , 2)
query7 = df1.reset_index().iloc[:,[0,1,-1]]
query7.head(10)

,country,num_students,fractionBelowRatio
0,Argentina,108373,0.00
1,Australia,713376,0.20
2,Austria,134477,0.45
3,Bangladesh,62716,1.00
4,Belarus,29303,0.00
5,Belgium,169661,0.00
6,Brazil,494251,0.56
7,Canada,623629,0.14
8,Chile,90951,1.00
9,China,1015150,0.87


## 8) Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [47]:
edu = pd.read_csv('educational_attainment_supplementary_data.csv')
edu = edu.dropna(subset=['country_name','series_name'])
print(edu.isna().sum()[['country_name','series_name']])
edu

country_name    0
series_name     0
dtype: int64


,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


## 9) From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [48]:
edu_unpivoted = edu.melt(id_vars=['country_name','series_name'], var_name='year', value_name='value')
edu_unpivoted

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


## 10) For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [49]:
query10 = pd.DataFrame(pd.concat([shanghai.university_name, times.university_name, cwur.institution], ignore_index=True).
                       dropna().drop_duplicates()).rename(columns={0: "university_name"}).set_index('university_name')
print('Number of Universities founded: ',len(query10))
query10.head()

Number of Universities founded:  1452


""
university_name
Harvard University
University of Cambridge
Stanford University
"University of California, Berkeley"
Massachusetts Institute of Technology (MIT)


In [50]:
query10['timesEntries'] = times.groupby(times.university_name).world_rank.count()
query10['cwurEntries'] = cwur.groupby(cwur.institution).world_rank.count()
query10['shanghaiEntries'] = shanghai.groupby(shanghai.university_name).world_rank.count()
query10['apperence'] = query10.count(axis=1)
query10 = query10.fillna(0)
query10

,timesEntries,cwurEntries,shanghaiEntries,apperence
university_name,,,,
Harvard University,6.0,4.0,11.0,3
University of Cambridge,6.0,4.0,11.0,3
Stanford University,6.0,4.0,11.0,3
"University of California, Berkeley",6.0,4.0,10.0,3
Massachusetts Institute of Technology (MIT),0.0,0.0,11.0,1
...,...,...,...,...
Shenzhen University,0.0,1.0,0.0,1
Tianjin Medical University,0.0,1.0,0.0,1
Babeș-Bolyai University,0.0,1.0,0.0,1


In [51]:
query10.apperence.value_counts()

1    766
3    362
2    324
Name: apperence, dtype: int64

## 11) In the times ranking, compute the number of times each university appears

In [57]:
query11 = times.groupby(times.university_name).university_name.count().reset_index(name='timesEntries')
query11

,university_name,timesEntries
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


## 12) Find the universities that appear at most twice in the times ranking.

In [53]:
query12 = query11[query11.timesEntries <= 2]
query12

,university_name,timesEntries
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


## 13) The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [54]:
shanWR = shanghai[['university_name','year','world_rank']].drop_duplicates()
timesWR = times[['university_name','year','world_rank']].drop_duplicates()
cwurWR = cwur[['institution','year','world_rank']].drop_duplicates().rename(columns={'institution' : 'university_name'})

In [55]:
timesWR.world_rank = timesWR.world_rank.apply(lambda x: RankToInt(x))
shanWR.world_rank = shanWR.world_rank.apply(lambda x: RankToInt(x))
allWR = shanWR.merge(timesWR, how='outer', on=['university_name','year']).merge(cwurWR, how='outer', on=['university_name','year'])
allWR

,university_name,year,world_rank_x,world_rank_y,world_rank
0,Harvard University,2005,1.0,NaN,NaN
1,University of Cambridge,2005,2.0,NaN,NaN
2,Stanford University,2005,3.0,NaN,NaN
3,"University of California, Berkeley",2005,4.0,NaN,NaN
4,Massachusetts Institute of Technology (MIT),2005,5.0,NaN,NaN
...,...,...,...,...,...
7784,University of the Algarve,2015,NaN,NaN,996.0
7785,Alexandria University,2015,NaN,NaN,997.0
7786,Federal University of Ceará,2015,NaN,NaN,998.0
7787,University of A Coruña,2015,NaN,NaN,999.0


**Prendo solo le righe senza Null e con un unico valore**

In [56]:
allWR[allWR.iloc[:,2:].nunique(axis=1) == 1].dropna()

,university_name,year,world_rank_x,world_rank_y,world_rank
3796,Stanford University,2013,2.0,2.0,2.0
